In [2]:
import kaggle
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from spacy.cli import download
from spacy import load
import time

In [3]:
kaggle.api.dataset_download_files('uciml/sms-spam-collection-dataset', path='./', unzip=True)

In [4]:
df = pd.read_csv('./spam.csv', encoding='latin-1')

In [5]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [6]:
nltk.download('wordnet')
nltk.download('wordnet2022')
nltk.download('omw-1.4')
download('en_core_web_sm')
nlp = load('en_core_web_sm')


[nltk_data] Downloading package wordnet to /home/querriqe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to
[nltk_data]     /home/querriqe/nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/querriqe/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 116.3 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)

In [8]:
df['v1'].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

In [9]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
df['v1'] = le.fit_transform(df['v1'])
df

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [10]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemmatizer = WordNetLemmatizer() #lemmatizer (cats -> cat, dogs -> dog, etc.)
stop_words = stopwords.words('english') #stopwords (is, at, etc.)
tokenizer = nltk.RegexpTokenizer(r'\w+') 

def token_list(massage):
    tokens = tokenizer.tokenize(massage)
    lc_tokens = [t.lower() for t in tokens] # lowercased tokens
    lm_tokens = [lemmatizer.lemmatize(t) for t in lc_tokens] #lemmatized tokens
    tokens = [t for t in lm_tokens if t not in stop_words] #get rid of stopwords(is, at, etc.)
    
    return tokens

In [11]:
df = df.sample(frac=1, random_state=42)
df = df.reset_index(drop=True)

train_test_rat = int(len(df) * 0.8)
train_df, test_df = df[:train_test_rat], df[train_test_rat:]

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df['v1'].value_counts()

v1
0    3867
1     590
Name: count, dtype: int64

In [12]:
token_counter = {}

for message in train_df['v2']:
    tokenlist = token_list(message)
    
    for token in tokenlist:
        if token in token_counter:
            token_counter[token] +=1
        else:
            token_counter[token] = 1

len(token_counter)

7080

In [13]:
token_counter

{'funny': 6,
 'fact': 11,
 'nobody': 7,
 'teach': 9,
 'volcano': 1,
 '2': 411,
 'erupt': 1,
 'tsunami': 2,
 'arise': 1,
 'hurricane': 1,
 'sway': 1,
 'aroundn': 1,
 '1': 143,
 'hw': 4,
 'choose': 17,
 'wife': 27,
 'natural': 2,
 'disaster': 2,
 'happens': 7,
 'sent': 50,
 'score': 4,
 'sophas': 1,
 'secondary': 1,
 'application': 2,
 'school': 20,
 'think': 110,
 'thinking': 16,
 'applying': 1,
 'research': 4,
 'cost': 32,
 'also': 56,
 'contact': 61,
 'joke': 10,
 'ogunrinde': 1,
 'one': 141,
 'le': 10,
 'expensive': 6,
 'know': 218,
 'someone': 56,
 'fancy': 15,
 'call': 501,
 '09058097218': 1,
 'find': 63,
 'pobox': 12,
 '6': 37,
 'ls15hb': 1,
 '150p': 56,
 'promise': 12,
 'getting': 41,
 'soon': 45,
 'text': 186,
 'morning': 69,
 'let': 75,
 'made': 23,
 'ok': 246,
 'congratulation': 10,
 'ur': 304,
 'awarded': 27,
 'either': 20,
 'å': 242,
 '500': 35,
 'cd': 10,
 'gift': 21,
 'voucher': 30,
 'free': 227,
 'entry': 21,
 '100': 35,
 'weekly': 21,
 'draw': 37,
 'txt': 139,
 'music': 

In [14]:
def keep_token(token,threshold):
    if token not in token_counter:
        return False
    else:
        return token_counter[token] > threshold

In [15]:
features = set()

for token in token_counter:
    if keep_token(token,100):
        features.add(token)

In [16]:
token_map = {t:i for t,i in zip(features, range(len(features)))}
token_map

{'reply': 0,
 'take': 1,
 'text': 2,
 'day': 3,
 'n': 4,
 'back': 5,
 'send': 6,
 'week': 7,
 '1': 8,
 'hi': 9,
 'one': 10,
 'k': 11,
 'txt': 12,
 '2': 13,
 'need': 14,
 'sorry': 15,
 'å': 16,
 'great': 17,
 'like': 18,
 'see': 19,
 'gt': 20,
 'tell': 21,
 'c': 22,
 'stop': 23,
 'want': 24,
 'good': 25,
 'please': 26,
 'ha': 27,
 'know': 28,
 'go': 29,
 '4': 30,
 'today': 31,
 'phone': 32,
 'wa': 33,
 'new': 34,
 'mobile': 35,
 'got': 36,
 'make': 37,
 'dont': 38,
 'u': 39,
 'home': 40,
 'still': 41,
 'time': 42,
 'lor': 43,
 'free': 44,
 'call': 45,
 'ok': 46,
 'come': 47,
 'ur': 48,
 'lt': 49,
 'love': 50,
 'da': 51,
 'going': 52,
 'think': 53,
 'r': 54,
 'later': 55,
 'get': 56}

In [17]:
def message_to_count_vector(message):
    count_vector = np.zeros(len(features))

    processed_list_of_tokens = token_list(message)

    for token in processed_list_of_tokens:
        if token not in features:
            continue
        index = token_map[token]
        count_vector[index] += 1
        
    return count_vector

In [18]:
def to_X_y(dff):
    y = dff['v1'].to_numpy().astype(int)
    
    mess_col = dff['v2']
    count_vectors = []
    
    for message in mess_col:
        count_vector = message_to_count_vector(message)
        count_vectors.append(count_vector)
        
    X= np.array(count_vectors).astype(int)
    
    return X, y

In [19]:
X_train, y_train = to_X_y(train_df)

X_test, y_test = to_X_y(test_df)

In [20]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train)

X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

In [21]:
from sklearn.model_selection import train_test_split 

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score
import time


init_models = { 'Logistic Regression' :  LogisticRegression(),
                'SVC': SVC(),
                'K-Neighbors Classifier': KNeighborsClassifier(),
                'Decision Tree Classifier': DecisionTreeClassifier(),
                'Random Forest Classifier' : RandomForestClassifier(),
                'Gradient Boosting Classifier': GradientBoostingClassifier(),
                'XGB Classifier' : XGBClassifier(),
                'Cat Boost' : CatBoostClassifier()
               }

timer=[]
acc = []
models_names = []
for i, (key,model) in enumerate(init_models.items()):
    start_time = time.time()
    model.fit(X_train, y_train)
    end_time = time.time()
    models_names.append(key)
    acc.append(np.mean(cross_val_score(model, X_train, y_train, cv=5)))
    timer.append(end_time-start_time)
models_scores = pd.DataFrame({'model name': models_names, 'cross-val accuracy score': acc, 'time': timer})
models_scores.head(8)

Learning rate set to 0.017728
0:	learn: 0.6715676	total: 54.3ms	remaining: 54.2s
1:	learn: 0.6489933	total: 55.3ms	remaining: 27.6s
2:	learn: 0.6280280	total: 56.2ms	remaining: 18.7s
3:	learn: 0.6090535	total: 57.2ms	remaining: 14.2s
4:	learn: 0.5936612	total: 58.1ms	remaining: 11.6s
5:	learn: 0.5753783	total: 59ms	remaining: 9.77s
6:	learn: 0.5576316	total: 59.9ms	remaining: 8.5s
7:	learn: 0.5427849	total: 60.8ms	remaining: 7.54s
8:	learn: 0.5252803	total: 61.7ms	remaining: 6.79s
9:	learn: 0.5128062	total: 62.6ms	remaining: 6.2s
10:	learn: 0.4984479	total: 63.5ms	remaining: 5.71s
11:	learn: 0.4837922	total: 64.4ms	remaining: 5.3s
12:	learn: 0.4704334	total: 65.5ms	remaining: 4.97s
13:	learn: 0.4585743	total: 66.4ms	remaining: 4.67s
14:	learn: 0.4473978	total: 67.3ms	remaining: 4.42s
15:	learn: 0.4360951	total: 68.2ms	remaining: 4.2s
16:	learn: 0.4273077	total: 69.1ms	remaining: 4s
17:	learn: 0.4183205	total: 70.1ms	remaining: 3.82s
18:	learn: 0.4066979	total: 71ms	remaining: 3.67s
19:

,model name,cross-val accuracy score,time
0,Logistic Regression,0.940533,0.016887
1,SVC,0.948948,0.158059
2,K-Neighbors Classifier,0.930996,0.000651
3,Decision Tree Classifier,0.941094,0.011311
4,Random Forest Classifier,0.954278,0.209444
5,Gradient Boosting Classifier,0.952314,0.196232
6,XGB Classifier,0.953436,0.047663
7,Cat Boost,0.955400,1.139947


In [23]:
from sklearn.model_selection import GridSearchCV

model = KNeighborsClassifier()
n_neighbors = [3,4,5,6]
leaf_size = [20,30,40,50]
params = { 'n_neighbors': n_neighbors, 'leaf_size': leaf_size }

boost_grid = GridSearchCV(estimator = model, param_grid = params, 
                                cv = 5, verbose=2, scoring='accuracy',n_jobs = -1)
boost_grid.fit(X_train,y_train)
print(boost_grid.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[CV] END ........................leaf_size=20, n_neighbors=3; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=3; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=3; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=3; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=4; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=3; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=4; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=4; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=4; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=6; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=6; total time=   0.0s
[CV] END ........................leaf_size=20, n_neighbors=6; total time=   0.0s
[CV] END ...................

In [24]:
from sklearn.metrics import classification_report

start_time = time.time()

model = KNeighborsClassifier(n_neighbors=3, leaf_size=20)
model.fit(X_train, y_train)
prediction = model.predict(X_val)

end_time = time.time()

print(classification_report(y_val, prediction), '\n')
print('time: ', end_time-start_time)    

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       775
           1       0.92      0.68      0.78       117

    accuracy                           0.95       892
   macro avg       0.94      0.83      0.87       892
weighted avg       0.95      0.95      0.95       892
 

time:  0.021932601928710938


In [25]:
KNN_test_df = pd.DataFrame({'message': test_df['v2'], 'actual': y_test, 'prediction': model.predict(X_test)})
KNN_test_df

,message,actual,prediction
0,Thank you. I like you as well...,0,0
1,"New Tones This week include: 1)McFly-All Ab..,...",1,0
2,I am not sure about night menu. . . I know onl...,0,0
3,Hope ur head doesn't hurt 2 much ! Am ploughin...,0,0
4,Hey what how about your project. Started aha da.,0,0
...,...,...,...
1110,I came hostel. I m going to sleep. Plz call me...,0,0
1111,"Sorry, I'll call later",0,0
1112,Prabha..i'm soryda..realy..frm heart i'm sory,0,0
1113,Nt joking seriously i told,0,0


In [26]:
model = RandomForestClassifier()
n_estimators = [50,100,200,500,1000]
max_depth = [None,1,2,3]
params = { 'n_estimators': n_estimators, 'max_depth': max_depth }

boost_grid = GridSearchCV(estimator = model, param_grid = params, 
                                cv = 5, verbose=2, scoring='accuracy',n_jobs = -1)
boost_grid.fit(X_train,y_train)
print(boost_grid.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ....................max_depth=None, n_estimators=50; total time=   0.2s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.4s
[CV] END ...................max_depth=None, n_estimators=100; total time=   0.4s
[CV] END .......................max_depth=1, n_estimators=50; total time=   0.1s
[CV] END .......................max_depth=1, n_

In [27]:
start_time = time.time()
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train)
prediction = model.predict(X_val)
end_time = time.time()

print(classification_report(y_val, prediction), '\n')
print('time: ', end_time-start_time)

              precision    recall  f1-score   support

           0       0.97      0.98      0.97       775
           1       0.83      0.82      0.83       117

    accuracy                           0.96       892
   macro avg       0.90      0.90      0.90       892
weighted avg       0.95      0.96      0.95       892
 

time:  0.44256067276000977


In [28]:
RFC_test_df = pd.DataFrame({'message': test_df['v2'], 'actual': y_test, 'prediction': model.predict(X_test)})

In [29]:
print(KNN_test_df.loc[KNN_test_df['actual'] != KNN_test_df['prediction']])
print(1 - (len(KNN_test_df.loc[KNN_test_df['actual'] != KNN_test_df['prediction']])/len(KNN_test_df)))

                                                message  actual  prediction
1     New Tones This week include: 1)McFly-All Ab..,...       1           0
10    PRIVATE! Your 2003 Account Statement for <fone...       1           0
16              Call me da, i am waiting for your call.       0           1
25    SMSSERVICES. for yourinclusive text credits, p...       1           0
27    BangBabes Ur order is on the way. U SHOULD rec...       1           0
...                                                 ...     ...         ...
966   Get your garden ready for summer with a FREE s...       1           0
999   How come it takes so little time for a child w...       1           0
1048  Bloomberg -Message center +447797706009 Why wa...       1           0
1056  You will be receiving this week's Triple Echo ...       1           0
1067  Fantasy Football is back on your TV. Go to Sky...       1           0

[65 rows x 3 columns]
0.9417040358744395


In [30]:
print(RFC_test_df.loc[RFC_test_df['actual'] != RFC_test_df['prediction']])
print(1 - (len(RFC_test_df.loc[RFC_test_df['actual'] != RFC_test_df['prediction']])/len(RFC_test_df)))

                                                message  actual  prediction
1     New Tones This week include: 1)McFly-All Ab..,...       1           0
21                       I'm in a movie. Call me 4 wat?       0           1
35             RCT' THNQ Adrian for U text. Rgds Vatian       1           0
56    You are now unsubscribed all services. Get ton...       1           0
66    Please CALL 08712402972 immediately as there i...       1           0
69    Bored housewives! Chat n date now! 0871750.77....       1           0
77    I take it the post has come then! You must hav...       0           1
88    You won't believe it but it's true. It's Incre...       1           0
104   Hello darlin ive finished college now so txt m...       0           1
115   LIFE has never been this much fun and great un...       1           0
117   Adult 18 Content Your video will be with you s...       1           0
138   Want explicit SEX in 30 secs? Ring 02073162414...       1           0
145   U were

**Summary**

I chosed fastest and most accurate models. 
- KNN: %94,17 accuracy within 0.141373872756958 seconds
- RFC: %95,15 accuracy within 0.44256067276000977 seconds
